In [18]:
import urllib.request
from tqdm import tqdm
import pandas as pd
import os
import pickle as pkl

from pydeseq2.dds import DeseqDataSet
from pydeseq2.default_inference import DefaultInference
from pydeseq2.ds import DeseqStats
from pydeseq2.utils import load_example_data

In [69]:
#loaf files

counts = pd.read_csv('GSE139580_Raw_counts.txt',sep='\t', )
metadata = pd.read_csv('SraRunTable.csv')
pd.DataFrame(counts) 
pd.DataFrame(metadata)
print(metadata)
print(counts)

            Run Assay Type  AvgSpotLen       Bases   BioProject     BioSample  \
0   SRR10363099    RNA-Seq          50  1076319800  PRJNA580367  SAMN13155854   
1   SRR10363100    RNA-Seq          50  1187457550  PRJNA580367  SAMN13155853   
2   SRR10363101    RNA-Seq          50  1051717150  PRJNA580367  SAMN13155852   
3   SRR10363102    RNA-Seq          50  1061309750  PRJNA580367  SAMN13155851   
4   SRR10363103    RNA-Seq          50  1177301550  PRJNA580367  SAMN13155850   
5   SRR10363104    RNA-Seq          50  1138573700  PRJNA580367  SAMN13155849   
6   SRR10363105    RNA-Seq          50  1008413950  PRJNA580367  SAMN13155848   
7   SRR10363106    RNA-Seq          50  1051331400  PRJNA580367  SAMN13155847   
8   SRR10363107    RNA-Seq          50   973003000  PRJNA580367  SAMN13155846   
9   SRR10363108    RNA-Seq          50  1047920250  PRJNA580367  SAMN13155845   
10  SRR10363109    RNA-Seq          50   966581850  PRJNA580367  SAMN13155844   
11  SRR10363110    RNA-Seq  

In [70]:
#set the index to the genes column  
counts = counts.set_index('gene_id')
counts

,KO_0dyn_rep1,KO_0dyn_rep2,KO_0dyn_rep3,KO_2dyn_rep1,KO_2dyn_rep2,KO_2dyn_rep3,KO_30dyn_rep1,KO_30dyn_rep2,KO_30dyn_rep3,WT_0dyn_rep1,WT_0dyn_rep2,WT_0dyn_rep3,WT_2dyn_rep1,WT_2dyn_rep2,WT_2dyn_rep3,WT_30dyn_rep1,WT_30dyn_rep2,WT_30dyn_rep3
gene_id,,,,,,,,,,,,,,,,,,
ENSMUSG00000025903,598,650,670,882,882,883,921,941,954,566,541,604,1162,1010,1117,1031,1189,1162
ENSMUSG00000033813,365,410,363,342,374,383,398,439,414,387,356,410,540,456,472,501,522,559
ENSMUSG00000033793,1437,1602,1445,2012,2053,2177,1601,1611,1501,708,705,817,1232,989,1071,917,955,1027
ENSMUSG00000025905,5,1,1,10,6,2,1,2,0,0,0,0,1,0,0,0,0,0
ENSMUSG00000025907,1190,1361,1208,1874,1922,1910,1742,1785,1697,1191,1085,1261,1266,1320,1396,1445,1680,1558
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSMUSG00000095867,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ENSMUSG00000094660,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ENSMUSG00000095650,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [71]:
#remove columns with zero counts  

counts = counts[counts.sum(axis=1) > 0]
counts

,KO_0dyn_rep1,KO_0dyn_rep2,KO_0dyn_rep3,KO_2dyn_rep1,KO_2dyn_rep2,KO_2dyn_rep3,KO_30dyn_rep1,KO_30dyn_rep2,KO_30dyn_rep3,WT_0dyn_rep1,WT_0dyn_rep2,WT_0dyn_rep3,WT_2dyn_rep1,WT_2dyn_rep2,WT_2dyn_rep3,WT_30dyn_rep1,WT_30dyn_rep2,WT_30dyn_rep3
gene_id,,,,,,,,,,,,,,,,,,
ENSMUSG00000025903,598,650,670,882,882,883,921,941,954,566,541,604,1162,1010,1117,1031,1189,1162
ENSMUSG00000033813,365,410,363,342,374,383,398,439,414,387,356,410,540,456,472,501,522,559
ENSMUSG00000033793,1437,1602,1445,2012,2053,2177,1601,1611,1501,708,705,817,1232,989,1071,917,955,1027
ENSMUSG00000025905,5,1,1,10,6,2,1,2,0,0,0,0,1,0,0,0,0,0
ENSMUSG00000025907,1190,1361,1208,1874,1922,1910,1742,1785,1697,1191,1085,1261,1266,1320,1396,1445,1680,1558
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSMUSG00000096768,39,29,40,31,57,43,65,75,59,213,202,263,220,176,177,160,184,183
ENSMUSG00000068457,67,72,78,97,82,104,49,77,60,608,527,704,596,601,622,516,592,547
ENSMUSG00000069045,169,202,173,225,221,201,163,190,152,1149,1022,1224,1198,1168,1323,1068,1246,1194


In [72]:
#pyDESeq requires counts matrix with rows as samples and genes as columns (transpose)  

counts = counts.T
counts.head(50)

gene_id,ENSMUSG00000025903,ENSMUSG00000033813,ENSMUSG00000033793,ENSMUSG00000025905,ENSMUSG00000025907,ENSMUSG00000087247,ENSMUSG00000033740,ENSMUSG00000051285,ENSMUSG00000061024,ENSMUSG00000025911,...,ENSMUSG00000056673,ENSMUSG00000069049,ENSMUSG00000095852,ENSMUSG00000096520,ENSMUSG00000095153,ENSMUSG00000096768,ENSMUSG00000068457,ENSMUSG00000069045,ENSMUSG00000069044,ENSMUSG00000095650
KO_0dyn_rep1,598,365,1437,5,1190,0,5,1296,454,68,...,69,76,0,0,0,39,67,169,2,0
KO_0dyn_rep2,650,410,1602,1,1361,0,5,1464,494,75,...,79,104,0,0,0,29,72,202,0,0
KO_0dyn_rep3,670,363,1445,1,1208,2,1,1366,482,76,...,67,76,0,0,0,40,78,173,0,0
KO_2dyn_rep1,882,342,2012,10,1874,1,3,1156,618,136,...,119,94,0,0,0,31,97,225,2,0
KO_2dyn_rep2,882,374,2053,6,1922,2,2,1168,608,129,...,110,80,0,0,1,57,82,221,0,0
KO_2dyn_rep3,883,383,2177,2,1910,1,1,1134,611,134,...,128,102,0,0,0,43,104,201,1,0
KO_30dyn_rep1,921,398,1601,1,1742,2,1,1042,820,20,...,83,71,0,0,0,65,49,163,2,0
KO_30dyn_rep2,941,439,1611,2,1785,2,0,1093,816,36,...,88,74,0,0,0,75,77,190,0,0
KO_30dyn_rep3,954,414,1501,0,1697,4,0,1024,810,31,...,75,64,0,0,0,59,60,152,1,0
WT_0dyn_rep1,566,387,708,0,1191,0,0,398,392,18,...,655,805,2,8,0,213,608,1149,0,0


In [76]:
#prepare metadata using the counts index

metadata_df = pd.DataFrame(zip(counts.index, [   'KO_0dyn', 'KO_0dyn', 'KO_0dyn',
    'KO_2dyn', 'KO_2dyn', 'KO_2dyn',
    'KO_30dyn', 'KO_30dyn', 'KO_30dyn',
    'WT_0dyn', 'WT_0dyn', 'WT_0dyn',
    'WT_2dyn', 'WT_2dyn', 'WT_2dyn',
    'WT_30dyn', 'WT_30dyn', 'WT_30dyn']), columns = ['Sample','Condition'])


In [77]:
#set the sample column as the index
metadata_df = metadata_df.set_index('Sample')
metadata_df

,Condition
Sample,
KO_0dyn_rep1,KO_0dyn
KO_0dyn_rep2,KO_0dyn
KO_0dyn_rep3,KO_0dyn
KO_2dyn_rep1,KO_2dyn
KO_2dyn_rep2,KO_2dyn
KO_2dyn_rep3,KO_2dyn
KO_30dyn_rep1,KO_30dyn
KO_30dyn_rep2,KO_30dyn
KO_30dyn_rep3,KO_30dyn


In [80]:
#DESeq data object  
inference = DefaultInference(n_cpus=8)
dds = DeseqDataSet(
    counts=counts,
    metadata=metadata_df,
    design_factors="Condition",
    refit_cooks=True,
    inference=inference,
    # n_cpus=8, # n_cpus can be specified here or in the inference object
)

/opt/anaconda3/envs/gene_expression/lib/python3.12/site-packages/pydeseq2/dds.py:270: UserWarning: Some factor levels in the design contain underscores ('_').
                    They will be converted to hyphens ('-').
  self.obsm["design_matrix"] = build_design_matrix(


In [86]:
#Run deseq2
dds.deseq2()

Fitting size factors...
... done in 0.07 seconds.

Fitting dispersions...
... done in 2.93 seconds.

Fitting dispersion trend curve...
... done in 0.45 seconds.

Fitting MAP dispersions...
... done in 3.79 seconds.

Fitting LFCs...
... done in 1.91 seconds.

Calculating cook's distance...
... done in 0.04 seconds.

Replacing 0 outlier genes.



In [97]:
#dds.obs.Condition
dds

AnnData object with n_obs × n_vars = 18 × 17710
    obs: 'Condition'
    uns: 'trend_coeffs', 'disp_function_type', '_squared_logres', 'prior_disp_var'
    obsm: 'design_matrix', 'size_factors', '_mu_LFC', '_hat_diagonals', 'replaceable'
    varm: '_normed_means', 'non_zero', '_MoM_dispersions', 'genewise_dispersions', '_genewise_converged', 'fitted_dispersions', 'MAP_dispersions', '_MAP_converged', 'dispersions', '_outlier_genes', 'LFC', '_LFC_converged', 'replaced', 'refitted', '_pvalue_cooks_outlier'
    layers: 'normed_counts', '_mu_hat', 'cooks'

In [96]:
# stats

stats_res = DeseqStats(dds, contrast=('Condition','KO-0dyn','WT-0dyn'))

In [98]:
stats_res.summary()

Running Wald tests...


Log2 fold change & Wald test p-value: Condition KO-0dyn vs WT-0dyn
                       baseMean  log2FoldChange     lfcSE       stat  \
gene_id                                                                
ENSMUSG00000025903   864.427007        0.319183  0.065789   4.851616   
ENSMUSG00000033813   423.257754        0.134358  0.075407   1.781759   
ENSMUSG00000033793  1347.132609        1.161564  0.055542  20.913158   
ENSMUSG00000025905     1.675489        3.826113  2.162003   1.769707   
ENSMUSG00000025907  1499.537286        0.240434  0.053206   4.518906   
...                         ...             ...       ...        ...   
ENSMUSG00000096768   117.485044       -2.491420  0.171255 -14.548007   
ENSMUSG00000068457   311.305417       -2.925411  0.118908 -24.602217   
ENSMUSG00000069045   639.096733       -2.488611  0.081095 -30.687760   
ENSMUSG00000069044     0.493184        2.028119  2.768312   0.732619   
ENSMUSG00000095650     0.049929       -0.810116  5.539875  -0.146234 

... done in 3.90 seconds.

